In [1]:
import os
from PIL import Image, ImageEnhance
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
def preprocess_image(image_path, size=(224, 224)):
    img = Image.open(image_path)
    
    # Quality enhancement
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(2.0)  # Increase sharpness
    
    # Noise reduction
    image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # Consistency
    img = img.resize(size, Image.LANCZOS)
    
    # Canny edge detection
    image = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    edges = cv2.Canny(image, 100, 200)
    img = Image.fromarray(edges)
    
    return img

# Apply preprocessing and save to new directory
input_dir = 'images'
output_dir = 'processed_images'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for category in os.listdir(input_dir):
    category_path = os.path.join(input_dir, category)
    if os.path.isdir(category_path):
        output_category_path = os.path.join(output_dir, category)
        if not os.path.exists(output_category_path):
            os.makedirs(output_category_path)
        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name)
            preprocessed_img = preprocess_image(image_path)
            preprocessed_img.save(os.path.join(output_category_path, image_name))


In [3]:
train_dir = 'images_train_test_val/train'
validation_dir = 'images_train_test_val/validation'

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 7350 images belonging to 21 classes.
Found 2100 images belonging to 21 classes.


In [4]:
base_model = EfficientNetB0(weights='imagenet', include_top=False)

2024-07-09 09:20:46.296783: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-07-09 09:20:46.296824: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-07-09 09:20:46.296847: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-07-09 09:20:46.296873: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-09 09:20:46.296897: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

In [6]:
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Define callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')


In [9]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[checkpoint, early_stopping]
)

# history = model.fit(
#     train_generator,
#     steps_per_epoch=(train_generator.samples // train_generator.batch_size) + int(train_generator.samples % train_generator.batch_size != 0),
#     validation_data=validation_generator,
#     validation_steps=(validation_generator.samples // validation_generator.batch_size) + int(validation_generator.samples % validation_generator.batch_size != 0),
#     epochs=10,
#     callbacks=[checkpoint, early_stopping]
# )

Epoch 1/10


2024-07-09 09:20:58.337939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/jay/anaconda3/envs/tf_m1_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


229/229 ━━━━━━━━━━━━━━━━━━━━ 2974s 13s/step - accuracy: 0.7293 - loss: 0.9293 - val_accuracy: 0.0500 - val_loss: 4.9545
Epoch 2/10
  1/229 ━━━━━━━━━━━━━━━━━━━━ 42:21 11s/step - accuracy: 1.0000 - loss: 0.1502

2024-07-09 10:10:34.736948: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-09 10:10:34.737198: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10907033679104508114
2024-07-09 10:10:34.737880: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16305375391206752604
2024-07-09 10:10:34.737902: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 18135339326618565806
2024-07-09 10:10:34.737915: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17273484836430136202
2024-07-09 10:10:34.738619: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12673635219111084338
2024-07-09 10:10:34.738635: I tensorflow/core/framework/local_rend

229/229 ━━━━━━━━━━━━━━━━━━━━ 32s 94ms/step - accuracy: 1.0000 - loss: 0.1502 - val_accuracy: 0.0000e+00 - val_loss: 4.8312
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9698s 42s/step - accuracy: 0.9107 - loss: 0.2959 - val_accuracy: 0.0587 - val_loss: 10.4338
Epoch 4/10
  1/229 ━━━━━━━━━━━━━━━━━━━━ 33:24 9s/step - accuracy: 0.9062 - loss: 0.1717

2024-07-09 12:52:42.990323: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-09 12:52:42.990642: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10907033679104508114
2024-07-09 12:52:42.990979: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12673635219111084338
2024-07-09 12:52:42.990995: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11276508908383765540
2024-07-09 12:52:42.991024: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 5282435617463010742
2024-07-09 12:52:42.991042: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2876645753785615778
2024-07-09 12:52:42.991059: I tensorflow/core/framework/local_rendez

229/229 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.9062 - loss: 0.1717 - val_accuracy: 0.1000 - val_loss: 9.9741
Epoch 5/10
 41/229 ━━━━━━━━━━━━━━━━━━━━ 40:48 13s/step - accuracy: 0.9345 - loss: 0.2256

In [ ]:
model.load_weights('best_model.keras')

train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(validation_generator)

print(f'Train accuracy: {train_acc:.4f}, Validation accuracy: {val_acc:.4f}')

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()